In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import skew
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
features = [c for c in train.columns if c not in['id','target']]

In [ ]:
train[features] = train[features].fillna(-1000)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
poly_matrix = poly.fit_transform(train[features])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(poly_matrix, train['target'], test_size=0.33, stratify=train['target'], random_state=42)

In [ ]:
%%time
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

In [ ]:
%%time
params = {}
# params['eta'] = 0.1
params['objective'] = 'binary:logistic'
# params['eval_metric'] = 'auc'
# params['max_depth'] = 2
# params['silent'] = 1
# params['learning_rate'] = 0.1
# params['subsample'] = 0.8
# params['colsample_bytree'] = 1
# params['colsample_bylevel'] = 1

watchlist = [(d_train, 'train'), (d_test, 'test')]
boost = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=100, verbose_eval=10)

In [ ]:
%%time
test['MSSubClass'] = test['MSSubClass'].astype(str)
test_cat_matrix = dv.transform(test[cats].T.to_dict().values())
test[nums] = test[nums].fillna(test[nums].mean())
test_poly = poly.transform(test[nums])
test_data_matrix = hstack([test_cat_matrix, test_poly])
d_test = xgb.DMatrix(test_data_matrix)

In [ ]:
preds = np.expm1(boost.predict(d_test))

In [ ]:
submission = pd.DataFrame()
submission['id'] = test.Id
submission['target'] = preds

In [ ]:
submission.to_csv('submission_xgb.csv', index=False)